In [ ]:
import h5py
import numpy as np
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
import h5py
import numpy as np
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt

import re
from collections import OrderedDict
import json

img = np.array(Image.open("./content/datasets/data_in/objects_testing/real/tub320x240_test/445_cam-image_array_.jpg"))
obj_ids = np.unique(img)
print(obj_ids)

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def load_data(images_folder):
  raw_image=[]
  gt=[]
  image_files = sorted(os.listdir(images_folder),key=natural_sort_key)
  for image_file in image_files:
      if image_file.split(".")[-1]=="jpg":
          image = cv2.imread(images_folder+image_file)
          raw_image.append(image)
      if image_file.split(".")[-1]=="json" and image_file!='meta.json':
          with open(images_folder+image_file, 'r') as json_file:
            data = json.load(json_file)
            user_angle = float(data.get('user/angle', None))
            gt.append(user_angle)
  return raw_image,gt


raw_image_sim = []
raw_image_real = []


print("-STARTING real- ")
images_folder = f"./content/datasets/data_in/objects_testing/real/tub320x240_test/"
raw_image_real,gt_real=load_data(images_folder)

print("-STARTING sim- ")
images_folder = f"./content/datasets/data_in/objects_testing/sim/tub320x240_test/"
raw_image_sim,gt_sim=load_data(images_folder)


print("-STARTING real- ")
images_folder = f"./content/datasets/data_in/objects_testing/real/tub320x240_train/"
raw_image_real_train,gt_real_train=load_data(images_folder)

print("-STARTING sim- ")
images_folder = f"./content/datasets/data_in/objects_testing/sim/tub320x240_train/"
raw_image_sim_train,gt_sim_train=load_data(images_folder)







In [ ]:
for i in range(0, 100,10):
      print(i)
      # Create a 2x2 grid of subplots
      fig, axs = plt.subplots(1, 2, figsize=(15, 6))
      plt.subplots_adjust(wspace=0.2, hspace=0.4)

      # Plot the real and fake images side by side in the first row
      axs[0].imshow(cv2.cvtColor(raw_image_real[i], cv2.COLOR_BGR2RGB))
      axs[0].set_title(str(gt_real[i]))
      axs[1].imshow(cv2.cvtColor(raw_image_sim[i], cv2.COLOR_BGR2RGB))
      axs[1].set_title(str(gt_sim[i]))
      for ax in axs.flat:
          ax.set_xticks([])
          ax.set_yticks([])

      # Show the plot for the current iteration
      plt.show()

for i in range(0, 100,1):
      print(i)
      # Create a 2x2 grid of subplots
      fig, axs = plt.subplots(1, 2, figsize=(15, 6))
      plt.subplots_adjust(wspace=0.2, hspace=0.4)

      # Plot the real and fake images side by side in the first row
      axs[0].imshow(cv2.cvtColor(raw_image_real_train[i], cv2.COLOR_BGR2RGB))
      axs[0].set_title(str(gt_real_train[i]))
      axs[1].imshow(cv2.cvtColor(raw_image_sim_train[i], cv2.COLOR_BGR2RGB))
      axs[1].set_title(str(gt_sim_train[i]))
      for ax in axs.flat:
          ax.set_xticks([])
          ax.set_yticks([])

      # Show the plot for the current iteration
      plt.show()

In [ ]:
dataset_index="0001"
def find_lowest_dimensions(images_list1, images_list2):
    # Find the lowest height and width across both lists
    lowest_height = min(images_list1[0].shape[0], images_list2[0].shape[0])
    lowest_width = min(images_list1[0].shape[1], images_list2[0].shape[1])
    return lowest_height, lowest_width

def crop_images_to_lowest_dimensions(images_list, lowest_height, lowest_width):
    cropped_images = [image[100:, :lowest_width,:] for image in images_list]
    return cropped_images

lowest_height=140
lowest_width=10000
height, width = find_lowest_dimensions(raw_image_real, raw_image_sim)
lowest_height=min(height,lowest_height)
lowest_width=min(width,lowest_width)
print(lowest_height,lowest_width)



In [5]:

loaded_dictionary_images_real={}
loaded_dictionary_images_sim={}

loaded_dictionary_gt_real={}
loaded_dictionary_gt_sim={}

loaded_dictionary_images_real[dataset_index] = crop_images_to_lowest_dimensions(raw_image_real, lowest_height, lowest_width)
loaded_dictionary_images_sim[dataset_index] = crop_images_to_lowest_dimensions(raw_image_sim, lowest_height, lowest_width)
loaded_dictionary_gt_real[dataset_index] = gt_real
loaded_dictionary_gt_sim[dataset_index] = gt_sim

loaded_dictionary_images_real_train={}
loaded_dictionary_images_sim_train={}

loaded_dictionary_gt_real_train={}
loaded_dictionary_gt_sim_train={}

loaded_dictionary_images_real_train[dataset_index] = crop_images_to_lowest_dimensions(raw_image_real_train, lowest_height, lowest_width)
loaded_dictionary_images_sim_train[dataset_index] = crop_images_to_lowest_dimensions(raw_image_sim_train, lowest_height, lowest_width)
loaded_dictionary_gt_real_train[dataset_index] = gt_real_train
loaded_dictionary_gt_sim_train[dataset_index] = gt_sim_train


train_indexes_gan = {}
test_indexes_gan = {}


pattern = 'vvvvvvvvvv'
pattern_pointer = 0
train_dataset_indices_inner = []
test_dataset_indices_inner = []

for index in range(0, min(len(loaded_dictionary_images_real[dataset_index]),len(loaded_dictionary_images_sim[dataset_index]))):
    test_dataset_indices_inner.append(index)

pattern = 'tttttttttt'
pattern_pointer = 0
for index in range(0, min(len(loaded_dictionary_images_real_train[dataset_index]),len(loaded_dictionary_images_sim_train[dataset_index]))):
    train_dataset_indices_inner.append(index)


train_indexes_gan[dataset_index]=train_dataset_indices_inner
test_indexes_gan[dataset_index]=test_dataset_indices_inner

print("GAN train and test")
print("Dataset", dataset_index)
print("Train: ",len(train_indexes_gan[dataset_index]))
print("Test: ",len(test_indexes_gan[dataset_index]))



GAN train and test
Dataset 0001
Train:  3657
Test:  262


In [ ]:
print(dataset_index)
for i in range(0, 10):
  # Create a 2x2 grid of subplots
  fig, axs = plt.subplots(1, 2, figsize=(15, 6))
  plt.subplots_adjust(wspace=0.2, hspace=0.4)

  # Plot the real and fake images side by side in the first row
  axs[0].imshow(cv2.cvtColor(loaded_dictionary_images_real[dataset_index][i], cv2.COLOR_BGR2RGB))
  axs[0].set_title(str(loaded_dictionary_gt_real[dataset_index][i]))
  axs[1].imshow(cv2.cvtColor(loaded_dictionary_images_sim[dataset_index][i], cv2.COLOR_BGR2RGB))
  axs[1].set_title(str(loaded_dictionary_gt_sim[dataset_index][i]))

  for ax in axs.flat:
      ax.set_xticks([])
      ax.set_yticks([])

  # Show the plot for the current iteration
  plt.show()

print(dataset_index)
for i in range(0, 10):
  # Create a 2x2 grid of subplots
  fig, axs = plt.subplots(1, 2, figsize=(15, 6))
  plt.subplots_adjust(wspace=0.2, hspace=0.4)

  # Plot the real and fake images side by side in the first row
  axs[0].imshow(cv2.cvtColor(loaded_dictionary_images_real_train[dataset_index][i], cv2.COLOR_BGR2RGB))
  axs[0].set_title(str(loaded_dictionary_gt_real_train[dataset_index][i]))
  axs[1].imshow(cv2.cvtColor(loaded_dictionary_images_sim_train[dataset_index][i], cv2.COLOR_BGR2RGB))
  axs[1].set_title(str(loaded_dictionary_gt_sim_train[dataset_index][i]))

  for ax in axs.flat:
      ax.set_xticks([])
      ax.set_yticks([])

  # Show the plot for the current iteration
  plt.show()

In [7]:
import json

In [ ]:
print(len(loaded_dictionary_images_real[dataset_index]))
print(len(loaded_dictionary_images_sim[dataset_index]))
print(len(loaded_dictionary_gt_real[dataset_index]))
print(len(loaded_dictionary_gt_sim[dataset_index]))

print(len(loaded_dictionary_images_real_train[dataset_index]))
print(len(loaded_dictionary_images_sim_train[dataset_index]))
print(len(loaded_dictionary_gt_real_train[dataset_index]))
print(len(loaded_dictionary_gt_sim_train[dataset_index]))

In [9]:
def save_sim_real_outputs(real_path,sim_path):
        dataset_index="0001"
        print(dataset_index)
        counter=0
        for i in range(0,len(loaded_dictionary_images_sim[dataset_index])):
                counter+=1
                input_image_sim = np.array(loaded_dictionary_images_sim[dataset_index][i])
                input_image_sim = tf.convert_to_tensor(input_image_sim.astype(np.uint8), np.uint8)
                input_image_sim = tf.reverse(input_image_sim, axis=[-1])
                png_image = tf.image.encode_png(input_image_sim)
                os.makedirs('./content/datasets/processed_data_testing/'+sim_path+'/', exist_ok=True)
                with open('./content/datasets/processed_data_testing/'+sim_path+'/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())
        counter=0
        for i in range(0,len(loaded_dictionary_images_real[dataset_index])):
                counter+=1
                input_image_real = np.array(loaded_dictionary_images_real[dataset_index][i])
                input_image_real = tf.convert_to_tensor(input_image_real.astype(np.uint8), np.uint8)
                input_image_real = tf.reverse(input_image_real, axis=[-1])
                
                png_image = tf.image.encode_png(input_image_real)
                os.makedirs('./content/datasets/processed_data_testing/'+real_path+'/', exist_ok=True)
                with open('./content/datasets/processed_data_testing/'+real_path+'/'+str(dataset_index)+'_'+str(i)+'.png', 'wb') as f:
                  f.write(png_image.numpy())


In [ ]:
save_sim_real_outputs("real","sim")